# Contact Fingerprint Detection

In [1]:
import os
import cv2
import random
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, regularizers, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### Defining utility functions

In [2]:
img_size = 96

def load_data(path, altered=True):
    """ Loads the fingerprint images.
    
    Args:
        path (str): path containing the fingerprint images
        altered (bool): altered or original fingerprint images (default=True)
        
    Returns:
        data (list): dataset of fingerprint images (user_id, fingernumber, image) 
    """
    
    print("Loading data from: ", path)
    data = []
    for img in os.listdir(path):
        imgname, ext = os.path.splitext(img)
        ID, etc = imgname.split('__')
        
        # Decrementing id by 1 (to_categorical encodes starting from 0)
        ID = int(ID) - 1
        
        # Extracting hand and finger information (altered images have additional info - type of modification)
        if altered:
            _, hand, finger, _, _ = etc.split('_')
        else:
            _, hand, finger, _  = etc.split('_')
        
        # Encoding fingers (left hand corresponding to 0-4 and right hand corresponding to 5-9)
        if hand=='Left':
            base = 0
        else: base  = 5
        if finger=="little":
            fingerNum = base + 0
        elif finger=='ring':
            fingerNum = base + 1
        elif finger=='middle':
            fingerNum = base + 2
        elif finger=='index':
            fingerNum = base + 3 
        else: fingerNum = base + 4
        
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        img_resize = cv2.resize(img_array, (img_size, img_size))
        data.append([ID, fingerNum, img_resize])
    return data

In [3]:
 def display_header(text):
    header = "-- " + text + " --"
    print("-"*len(header))
    print(header)
    print("-"*len(header)+"\n")

## 1. Loading fingerprint data
___

We have 4 folders containing fingerprint images:
- Altered-Easy
- Altered-Medium
- Altered-Hard
- Real

Let's load each one of them using the `load_data` function:

In [4]:
altered_images_path = "../data/SOCOFing/Altered/Altered-"
real_images_path = "../data/SOCOFing/Real"

# Loading the altered data
easy_data = load_data(altered_images_path+'Easy', altered=True)
medium_data = load_data(altered_images_path+'Medium', altered=True)
hard_data = load_data(altered_images_path+'Hard', altered=True)

# Concatenating altered data
altered_data = np.concatenate([easy_data, medium_data, hard_data], axis=0, dtype="object")

# Freeing memory
del easy_data, medium_data, hard_data

# Loading the real data
real_data = np.array(load_data(real_images_path, altered=False), dtype="object")

Loading data from:  ../data/SOCOFing/Altered/Altered-Easy
Loading data from:  ../data/SOCOFing/Altered/Altered-Medium
Loading data from:  ../data/SOCOFing/Altered/Altered-Hard


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Loading data from:  ../data/SOCOFing/Real


In [5]:
display_header("Number of samples")
print(f"Altered data: {len(altered_data)}")
print(f"Real data:    {len(real_data)}")

-----------------------
-- Number of samples --
-----------------------

Altered data: 49270
Real data:    6000


In [9]:
# Counting unique users
num_classes = len(np.unique(real_data[:, 0]))
print(f"There are {num_classes} unique user_ids")

NameError: name 'real_data' is not defined

## 2. Creating the Datasets
___

Here, we'll create datasets for training, validating and testing our model.

The altered data will be used for training and validating, and the real data will be used for testing.

- **Creating the training and validation datasets:**

In [7]:
X_altered, y_subjectID_altered = [], []

for subjectID, _, feature in altered_data:
    X_altered.append(feature)
    y_subjectID_altered.append(subjectID)

X_altered = np.array(X_altered).reshape(-1, img_size, img_size, 1)
X_altered = X_altered / 255.0  # Normalizing to [0, 1] 
y_subjectID_altered = to_categorical(y_subjectID_altered, num_classes=num_classes)

X_train, X_val, y_train, y_val = train_test_split(X_altered, y_subjectID_altered, test_size=0.2, random_state=2)

# Freeing memory
del altered_data, X_altered, y_subjectID_altered

- **Creating the testing dataset:**

In [8]:
# Creating the testing dataset
X_test, y_test = [], []

for subjectID, _, feature in real_data:
    X_test.append(feature)
    y_test.append(subjectID)

X_test = np.array(X_test).reshape(-1, img_size, img_size, 1)
X_test = X_test / 255.0

y_test = to_categorical(y_test, num_classes=num_classes)

# Freeing memory
del real_data

- **Visualizing data split:**

In [49]:
print(" Split           Feature shape      label shape")
print("------------------------------------------------")
print(f" Train:       {str(X_train.shape):<18}    {y_train.shape}")
print(f" Validation:  {str(X_val.shape):<18}    {y_val.shape}")
print(f" Test:        {str(X_test.shape):<18}    {y_test.shape}")

 Split           Feature shape      label shape
------------------------------------------------
 Train:       (39416, 96, 96, 1)    (39416, 600)
 Validation:  (9854, 96, 96, 1)     (9854, 600)
 Test:        (6000, 96, 96, 1)     (6000, 600)


## 3. Model
___